# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、選好予測のためのモデルを構築することを目的としています。具体的には、大規模言語モデル（LLM）を使用し、ユーザーの好みに基づいて2つの応答からどちらが好まれるかを予測するタスクに取り組んでいます。

Notebookでは、以下の手法やライブラリが使用されています：

1. **ライブラリのインストールとインポート**：
   - `peft`と`bitsandbytes`を利用し、特定のモデルやデータ処理に必要な機能を提供しています。
   - `pandas`を使用してデータを操作し、`datasets`ライブラリからデータを読み込んでいます。
   - `transformers`ライブラリからの様々なクラス（`AutoTokenizer`, `AutoModelForSequenceClassification`, `TrainingArguments`, `Trainer`など）を用いてモデルの構築とトレーニングを行っています。
   - `scikit-learn`から`log_loss`をインポートし、モデルの性能評価に使用しています。

2. **データ準備**：
   - トレーニングデータとして`train.csv`を読み込み、ベースとなるデータフレームを作成します。
   - 各サンプルに最も高い値を持つ応答をラベルとして追加し、データセットを必要な列のみに整理します。
   - データセットを5%の割合でトレーニングセットとテストセットに分割します。

3. **モデルの設定**：
   - `DeBERTa`モデルのファインチューニングに使用するチェックポイントを指定し、トークナイザーを初期化します。
   - `LoRa`（Low-Rank Adaptation）を利用して、効率的にモデルを訓練する設定を行います。

4. **トークナイゼーション**：
   - 各サンプルのプロンプトと応答をトークン化し、モデルが受け取れる形式に変換します。

5. **トレーニング**：
   - `TrainingArguments`を設定し、トレーニングの実行を管理する`Trainer`オブジェクトを初期化します。
   - ログロスや精度を評価指標として計算する関数を定義し、トレーニングを実行します。
   - トレーニング終了後、モデルとトークナイザーを保存します。

このNotebookは、LLMの強化学習タスクにおけるユーザーの選好を予測する機械学習モデルを構築し、データ準備からトレーニングまでの一連の流れを示しています。重要なポイントとして、再現性を考慮したシード値の設定や、最適化されたトレーニング戦略を採用していることが挙げられます。

---


# 用語概説 
以下は、初心者がつまずきそうな専門用語の簡単な解説です。ノートブック特有の用語や実務経験がないと馴染みが薄いかもしれないものに焦点を当てています。

### 専門用語の解説

1. **peft (Parameter-Efficient Fine-Tuning)**
   - モデルの微調整が必要な場合に、全てのパラメータを更新するのではなく、一部のパラメータだけを効率的に調整するための手法。特に巨大なモデルの微調整において、有用性がある。

2. **Lora (Low-Rank Adaptation)**
   - モデルの特定の部分（例えば、重み行列）を低ランクの行列で近似することにより、微調整を行う手法。これにより、パラメータ数を減らし、計算リソースを節約することができる。

3. **LoftQ**
   - モデルの量子化技術の一つ。通常、大きなモデルを小型化し、限られたリソース環境でのトレーニングや推論を可能にするために利用されます。LoftQは特に4ビットの量子化設定に言及しています。

4. **EarlyStoppingCallback**
   - モデルのトレーニング中に、監視している評価指標が改善しない場合にトレーニングを早期に停止するための機能。過学習を防ぎ、トレーニング時間を短縮するのに役立ちます。

5. **log_loss (対数損失)**
   - 分類モデルの性能を評価する指標の一つ。モデルが出す確率と実際のラベルとの間の対数的な差を測定します。小さい値が良いモデルを示す。特に確率的な出力を行うモデルで重要な評価基準です。

6. **DataCollatorWithPadding**
   - 異なる長さの入力シーケンスを持つバッチを整理し、モデルに入力する際に必要なパディングを追加するツール。これにより、バッチ内のすべての例が同じ長さを持つことが保証され、効率的に処理されます。

7. **Trainer**
   - Hugging FaceのTransformersライブラリにおけるトレーニングの高レベルAPI。モデル、データセット、トレーニング引数を渡すことにより、トレーニングプロセスを簡単に管理、実行できます。訓練フェーズの多くの複雑さを抽象化します。

8. **wandb (Weights and Biases)**
   - 機械学習の実験管理ツール。モデルのトレーニングプロセスの監視、ハイパーパラメータの管理、視覚化を行うために使用され、実験の再現性を高めるのに役立ちます。

これらは、ノートブックの内容を理解するにあたり、初心者が注意を向けるべき専門用語です。これにより、各用語が持つ意味とその用途が具体的に理解できるようになることを目指しています。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Libraries

</div>
<div class="column-right">

# 日本語訳

# ライブラリ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install peft bitsandbytes
```

</div>
<div class="column-right">

# 日本語訳

```python
# peftとbitsandbytesというライブラリをインストールします。
# これらのライブラリは、特定のモデルやデータ処理に必要な機能を提供します。
!pip install peft bitsandbytes
```

</div>
</details>

In [ ]:
# peftとbitsandbytesというライブラリをインストールします。
# これらのライブラリは、特定のモデルやデータ処理に必要な機能を提供します。
!pip install peft bitsandbytes

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
from datasets import Dataset, load_metric, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    AutoConfig
)

from pathlib import Path
from sklearn.metrics import log_loss
import numpy as np
import json

from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType, LoftQConfig
import bitsandbytes as bnb

from tqdm.auto import tqdm
import random
```

</div>
<div class="column-right">

# 日本語訳

```python
# pandasライブラリをインポートします。データの操作や分析を行うために使用します。
import pandas as pd

# datasetsライブラリから、Dataset、load_metric、load_datasetをインポートします。
# これらはデータの読み込みと評価指標の計算に使います。
from datasets import Dataset, load_metric, load_dataset

# transformersライブラリから、いくつかの重要なクラスをインポートします。
# AutoTokenizerはトークナイザーを自動的にロードするため、AutoModelForSequenceClassificationはシーケンス分類のモデルをロードするために使います。
# TrainingArgumentsはトレーニング設定を格納し、Trainerはトレーニングの実行を管理します。
# DataCollatorWithPaddingはバッチ内のデータを整形するために、EarlyStoppingCallbackはトレーニングの早期停止を設定します。
# AutoConfigはモデルの設定を自動的に取得するために使います。
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    AutoConfig
)

# pathlibライブラリをインポートします。ファイルパスの操作を簡単に行うために使用します。
from pathlib import Path

# sklearn.metricsからlog_lossをインポートします。予測の精度を評価するために使用します。
from sklearn.metrics import log_loss

# numpyライブラリをインポートします。数値計算を行うために使います。
import numpy as np

# jsonライブラリをインポートします。JSONデータの読み書きに使用します。
import json

# peftライブラリから、複数のクラスをインポートします。
# これらは特定のモデル設定やトレーニングを行うために使います。
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType, LoftQConfig

# bitsandbytesライブラリをインポートします。効率的な計算を行うために利用します。
import bitsandbytes as bnb

# tqdmライブラリを自動的にインポートします。プログレスバーを表示するために使います。
from tqdm.auto import tqdm

# randomライブラリをインポートします。ランダムな数値生成を行うために使います。
import random
```

</div>
</details>

In [ ]:
# pandasライブラリをインポートします。データの操作や分析を行うために使用します。
import pandas as pd

# datasetsライブラリから、Dataset、load_metric、load_datasetをインポートします。
# これらはデータの読み込みと評価指標の計算に使います。
from datasets import Dataset, load_metric, load_dataset

# transformersライブラリから、いくつかの重要なクラスをインポートします。
# AutoTokenizerはトークナイザーを自動的にロードするため、AutoModelForSequenceClassificationはシーケンス分類のモデルをロードするために使います。
# TrainingArgumentsはトレーニング設定を格納し、Trainerはトレーニングの実行を管理します。
# DataCollatorWithPaddingはバッチ内のデータを整形するために、EarlyStoppingCallbackはトレーニングの早期停止を設定します。
# AutoConfigはモデルの設定を自動的に取得するために使います。
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    AutoConfig
)

# pathlibライブラリをインポートします。ファイルパスの操作を簡単に行うために使用します。
from pathlib import Path

# sklearn.metricsからlog_lossをインポートします。予測の精度を評価するために使用します。
from sklearn.metrics import log_loss

# numpyライブラリをインポートします。数値計算を行うために使います。
import numpy as np

# jsonライブラリをインポートします。JSONデータの読み書きに使用します。
import json

# peftライブラリから、複数のクラスをインポートします。
# これらは特定のモデル設定やトレーニングを行うために使います。
from peft import get_peft_config, get_peft_model, PeftModel, PeftConfig, LoraConfig, TaskType, LoftQConfig

# bitsandbytesライブラリをインポートします。効率的な計算を行うために利用します。
import bitsandbytes as bnb

# tqdmライブラリを自動的にインポートします。プログレスバーを表示するために使います。
from tqdm.auto import tqdm

# randomライブラリをインポートします。ランダムな数値生成を行うために使います。
import random

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)
```

</div>
<div class="column-right">

# 日本語訳

```python
# kaggle_secretsライブラリからUserSecretsClientをインポートします。
# これにより、Kaggleのユーザーシークレットにアクセスできるようになります。
from kaggle_secrets import UserSecretsClient

# wandbライブラリをインポートします。これはWeights and Biasesを使用して実験を管理するためのライブラリです。
import wandb

# UserSecretsClientのインスタンスを作成します。このクライアントを使ってシークレット情報にアクセスします。
user_secrets = UserSecretsClient()

# "wandb_api_key"という名前のシークレット情報を取得します。これにはWandBのAPIキーが保存されています。
my_secret = user_secrets.get_secret("wandb_api_key") 

# 取得したAPIキーを使用してWandBにログインします。このステップが成功すると、WandBの機能が利用可能になります。
wandb.login(key=my_secret)
```

</div>
</details>

In [ ]:
# kaggle_secretsライブラリからUserSecretsClientをインポートします。
# これにより、Kaggleのユーザーシークレットにアクセスできるようになります。
from kaggle_secrets import UserSecretsClient

# wandbライブラリをインポートします。これはWeights and Biasesを使用して実験を管理するためのライブラリです。
import wandb

# UserSecretsClientのインスタンスを作成します。このクライアントを使ってシークレット情報にアクセスします。
user_secrets = UserSecretsClient()

# "wandb_api_key"という名前のシークレット情報を取得します。これにはWandBのAPIキーが保存されています。
my_secret = user_secrets.get_secret("wandb_api_key") 

# 取得したAPIキーを使用してWandBにログインします。このステップが成功すると、WandBの機能が利用可能になります。
wandb.login(key=my_secret)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%env WANDB_LOG_MODEL="checkpoint"
%env WANDB_PROJECT=LMSYS
```

</div>
<div class="column-right">

# 日本語訳

```python
# 環境変数WANDB_LOG_MODELを"checkpoint"に設定します。
# これはWandB（Weights and Biases）がモデルのチェックポイントをログに記録することを指示します。
%env WANDB_LOG_MODEL="checkpoint"

# 環境変数WANDB_PROJECTを"LMSYS"に設定します。
# これはWandBでのプロジェクト名を指定し、実験をこのプロジェクトに関連付けます。
%env WANDB_PROJECT=LMSYS
```

</div>
</details>

In [ ]:
# 環境変数WANDB_LOG_MODELを"checkpoint"に設定します。
# これはWandB（Weights and Biases）がモデルのチェックポイントをログに記録することを指示します。
%env WANDB_LOG_MODEL="checkpoint"

# 環境変数WANDB_PROJECTを"LMSYS"に設定します。
# これはWandBでのプロジェクト名を指定し、実験をこのプロジェクトに関連付けます。
%env WANDB_PROJECT=LMSYS

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Data & Configs

</div>
<div class="column-right">

# 日本語訳

# データと設定

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def seed_everything(seed):
    import random
    import os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(2024)
```

</div>
<div class="column-right">

# 日本語訳

```python
# すべてのランダム数生成器に同じシード値を設定する関数を定義します。
# この関数を使用することで、結果を再現可能にし、実験の一貫性を保ちます。
def seed_everything(seed):
    # randomライブラリをインポートします。Pythonのランダム生成器を操作するために使用します。
    import random
    # osライブラリをインポートします。オペレーティングシステムの機能にアクセスするために使用します。
    import os
    # numpyライブラリをインポートします。数値計算の際に使用します。
    import numpy as np
    # torchライブラリをインポートします。PyTorchを使った深層学習のために使用します。
    import torch
    
    # Pythonのランダム生成器のシード値を設定します。
    random.seed(seed)
    # 環境変数PYTHONHASHSEEDをシード値に設定します。これにより、ハッシュの一貫性が保たれます。
    os.environ['PYTHONHASHSEED'] = str(seed)
    # numpyのランダム生成器のシード値を設定します。
    np.random.seed(seed)
    # PyTorchのCPU用のランダム生成器のシード値を設定します。
    torch.manual_seed(seed)
    # PyTorchのGPU用のランダム生成器のシード値を設定します。
    torch.cuda.manual_seed(seed)
    # cuDNNの決定論的モードを有効にします。これにより、同じ入力に対して常に同じ出力が得られます。
    torch.backends.cudnn.deterministic = True
    
# 2024をシード値にして、seed_everything関数を呼び出します。これによってランダム生成器が初期化されます。
seed_everything(2024)
```

</div>
</details>

In [ ]:
# すべてのランダム数生成器に同じシード値を設定する関数を定義します。
# この関数を使用することで、結果を再現可能にし、実験の一貫性を保ちます。
def seed_everything(seed):
    # randomライブラリをインポートします。Pythonのランダム生成器を操作するために使用します。
    import random
    # osライブラリをインポートします。オペレーティングシステムの機能にアクセスするために使用します。
    import os
    # numpyライブラリをインポートします。数値計算の際に使用します。
    import numpy as np
    # torchライブラリをインポートします。PyTorchを使った深層学習のために使用します。
    import torch
    
    # Pythonのランダム生成器のシード値を設定します。
    random.seed(seed)
    # 環境変数PYTHONHASHSEEDをシード値に設定します。これにより、ハッシュの一貫性が保たれます。
    os.environ['PYTHONHASHSEED'] = str(seed)
    # numpyのランダム生成器のシード値を設定します。
    np.random.seed(seed)
    # PyTorchのCPU用のランダム生成器のシード値を設定します。
    torch.manual_seed(seed)
    # PyTorchのGPU用のランダム生成器のシード値を設定します。
    torch.cuda.manual_seed(seed)
    # cuDNNの決定論的モードを有効にします。これにより、同じ入力に対して常に同じ出力が得られます。
    torch.backends.cudnn.deterministic = True
    
# 2024をシード値にして、seed_everything関数を呼び出します。これによってランダム生成器が初期化されます。
seed_everything(2024)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
max_len = 1024
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最大長（max_len）を1024に設定します。
# これは、モデルに入力するシーケンスの最大トークン数を指定します。
# この長さを超えるトークンは切り捨てられるか、適切な処理が行われます。
max_len = 1024
```

</div>
</details>

In [ ]:
# 最大長（max_len）を1024に設定します。
# これは、モデルに入力するシーケンスの最大トークン数を指定します。
# この長さを超えるトークンは切り捨てられるか、適切な処理が行われます。
max_len = 1024

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Generate label & remove columns

</div>
<div class="column-right">

# 日本語訳

## ラベルの生成と列の削除

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
folder_path = Path('/kaggle/input/lmsys-chatbot-arena')
```

</div>
<div class="column-right">

# 日本語訳

```python
# フォルダーパスを設定します。
# '/kaggle/input/lmsys-chatbot-arena'はデータが格納されているディレクトリのパスです。
folder_path = Path('/kaggle/input/lmsys-chatbot-arena')
```

</div>
</details>

In [ ]:
# フォルダーパスを設定します。
# '/kaggle/input/lmsys-chatbot-arena'はデータが格納されているディレクトリのパスです。
folder_path = Path('/kaggle/input/lmsys-chatbot-arena')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_df = load_dataset('csv', data_files=str(folder_path / 'train.csv'))
# train_df = load_dataset('csv', data_files='external_train.csv')
# train_df = load_dataset('csv', data_files='/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv')
test_df = load_dataset('csv', data_files=str(folder_path / 'test.csv'))
sample_df = load_dataset('csv', data_files=str(folder_path / 'sample_submission.csv'))
```

</div>
<div class="column-right">

# 日本語訳

```python
# 'train.csv'ファイルを読み込み、データセットとしてtrain_dfに格納します。
# load_dataset関数を使ってCSV形式でデータを読み込みます。
train_df = load_dataset('csv', data_files=str(folder_path / 'train.csv'))

# 以下の行は以前のデータセットを読み込むためのコメントアウトされた例です。
# 'external_train.csv'という別のトレーニングデータを読み込む場合。
# train_df = load_dataset('csv', data_files='external_train.csv')
# '/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv'からのデータを読み込む場合。
# train_df = load_dataset('csv', data_files='/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv')

# 'test.csv'ファイルを読み込み、データセットとしてtest_dfに格納します。
test_df = load_dataset('csv', data_files=str(folder_path / 'test.csv'))

# 'sample_submission.csv'ファイルを読み込み、データセットとしてsample_dfに格納します。
sample_df = load_dataset('csv', data_files=str(folder_path / 'sample_submission.csv'))
```

</div>
</details>

In [ ]:
# 'train.csv'ファイルを読み込み、データセットとしてtrain_dfに格納します。
# load_dataset関数を使ってCSV形式でデータを読み込みます。
train_df = load_dataset('csv', data_files=str(folder_path / 'train.csv'))

# 以下の行は以前のデータセットを読み込むためのコメントアウトされた例です。
# 'external_train.csv'という別のトレーニングデータを読み込む場合。
# train_df = load_dataset('csv', data_files='external_train.csv')
# '/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv'からのデータを読み込む場合。
# train_df = load_dataset('csv', data_files='/kaggle/input/lmsys-additional-33k-labelled-conversations/lmsys-33k-deduplicated.csv')

# 'test.csv'ファイルを読み込み、データセットとしてtest_dfに格納します。
test_df = load_dataset('csv', data_files=str(folder_path / 'test.csv'))

# 'sample_submission.csv'ファイルを読み込み、データセットとしてsample_dfに格納します。
sample_df = load_dataset('csv', data_files=str(folder_path / 'sample_submission.csv'))

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def get_label(sample):
    sample['label'] = np.argmax([sample['winner_model_a'], sample['winner_model_b'], sample['winner_tie']])
    return sample
train_df = train_df.map(get_label)
```

</div>
<div class="column-right">

# 日本語訳

```python
# ラベルを生成する関数get_labelを定義します。
# この関数は、各サンプルに対して最も高い値を持つインデックスをラベルとして追加します。
def get_label(sample):
    # winner_model_a、winner_model_b、winner_tieの値の中で最大値を持つインデックスを取得し、labelとして設定します。
    sample['label'] = np.argmax([sample['winner_model_a'], sample['winner_model_b'], sample['winner_tie']])
    return sample

# train_dfデータセットにget_label関数を適用します。
# これにより、各サンプルにラベルが追加されます。
train_df = train_df.map(get_label)
```

</div>
</details>

In [ ]:
# ラベルを生成する関数get_labelを定義します。
# この関数は、各サンプルに対して最も高い値を持つインデックスをラベルとして追加します。
def get_label(sample):
    # winner_model_a、winner_model_b、winner_tieの値の中で最大値を持つインデックスを取得し、labelとして設定します。
    sample['label'] = np.argmax([sample['winner_model_a'], sample['winner_model_b'], sample['winner_tie']])
    return sample

# train_dfデータセットにget_label関数を適用します。
# これにより、各サンプルにラベルが追加されます。
train_df = train_df.map(get_label)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_df = train_df.select_columns(['prompt', 'response_a', 'response_b', 'label'])
train_df
```

</div>
<div class="column-right">

# 日本語訳

```python
# train_dfデータセットから必要な列だけを選択し、新しいデータフレームを作成します。
# ここでは、'prompt'、'response_a'、'response_b'、'label'の4つの列を選択します。
train_df = train_df.select_columns(['prompt', 'response_a', 'response_b', 'label'])

# 現在のtrain_dfデータセットを表示します。
# これにより、選択した列のみが含まれるデータフレームが確認できます。
train_df
```

</div>
</details>

In [ ]:
# train_dfデータセットから必要な列だけを選択し、新しいデータフレームを作成します。
# ここでは、'prompt'、'response_a'、'response_b'、'label'の4つの列を選択します。
train_df = train_df.select_columns(['prompt', 'response_a', 'response_b', 'label'])

# 現在のtrain_dfデータセットを表示します。
# これにより、選択した列のみが含まれるデータフレームが確認できます。
train_df

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
dataset = train_df['train'].train_test_split(0.05)
dataset
```

</div>
<div class="column-right">

# 日本語訳

```python
# train_dfデータセットを5%の割合でトレーニングセットとテストセットに分割します。
# train_test_split関数を使用して、データセットをシャッフルし、指定した割合で分割を行います。
dataset = train_df['train'].train_test_split(0.05)

# 現在のデータセットの分割結果を表示します。
# これにより、トレーニングセットとテストセットの構造を確認できます。
dataset
```

</div>
</details>

In [ ]:
# train_dfデータセットを5%の割合でトレーニングセットとテストセットに分割します。
# train_test_split関数を使用して、データセットをシャッフルし、指定した割合で分割を行います。
dataset = train_df['train'].train_test_split(0.05)

# 現在のデータセットの分割結果を表示します。
# これにより、トレーニングセットとテストセットの構造を確認できます。
dataset

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Models

</div>
<div class="column-right">

# 日本語訳

# モデル

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# model_name = "microsoft/deberta-v3-base"
adapther_tokenizer_path = "/kaggle/input/deberta-finetuned/DeBerta-base-5"
model_name = "/kaggle/input/deberta-finetuned/LMSYS/checkpoint-2980"
```

</div>
<div class="column-right">

# 日本語訳

```python
# 使用するモデルの名前を指定するための変数model_nameを定義します。
# このコメントアウトされた行は、microsoftのDeBERTa V3ベースモデルを指定するための例です。
# model_name = "microsoft/deberta-v3-base"

# アダプタートークナイザーのパスを指定します。
# このパスは、事前にファインチューニングされたDeBERTaモデルのトークナイザーが保存されている場所です。
adapther_tokenizer_path = "/kaggle/input/deberta-finetuned/DeBerta-base-5"

# 使用するモデルのチェックポイントのパスを指定します。
# これは、ファインチューニングされたモデルの保存場所を示しています。
model_name = "/kaggle/input/deberta-finetuned/LMSYS/checkpoint-2980"
```

</div>
</details>

In [ ]:
# 使用するモデルの名前を指定するための変数model_nameを定義します。
# このコメントアウトされた行は、microsoftのDeBERTa V3ベースモデルを指定するための例です。
# model_name = "microsoft/deberta-v3-base"

# アダプタートークナイザーのパスを指定します。
# このパスは、事前にファインチューニングされたDeBERTaモデルのトークナイザーが保存されている場所です。
adapther_tokenizer_path = "/kaggle/input/deberta-finetuned/DeBerta-base-5"

# 使用するモデルのチェックポイントのパスを指定します。
# これは、ファインチューニングされたモデルの保存場所を示しています。
model_name = "/kaggle/input/deberta-finetuned/LMSYS/checkpoint-2980"

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = AutoTokenizer.from_pretrained(adapther_tokenizer_path, model_max_length=max_len)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 指定したアダプタートークナイザーのパスからトークナイザーを初期化します。
# max_lenで定義した最大長に基づいて、トークナイザーが設定されます。
tokenizer = AutoTokenizer.from_pretrained(adapther_tokenizer_path, model_max_length=max_len)

# 指定したモデル名からシーケンス分類モデルを初期化します。
# num_labelsは分類するラベルの数を示し、ここでは3つのラベルがあることを指定します。
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
)
```

</div>
</details>

In [ ]:
# 指定したアダプタートークナイザーのパスからトークナイザーを初期化します。
# max_lenで定義した最大長に基づいて、トークナイザーが設定されます。
tokenizer = AutoTokenizer.from_pretrained(adapther_tokenizer_path, model_max_length=max_len)

# 指定したモデル名からシーケンス分類モデルを初期化します。
# num_labelsは分類するラベルの数を示し、ここでは3つのラベルがあることを指定します。
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
loftq_config = LoftQConfig(loftq_bits=4)
peft_config = LoraConfig(
#     target_modules=['query_proj', 'value_proj', 'key_proj'],
    use_rslora=True,
    r=16,
    lora_alpha=8,
    lora_dropout=0.1,
    task_type=TaskType.SEQ_CLS,
    init_lora_weights='loftq',
)

model = get_peft_model(model, peft_config)
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoftQConfigを使用して、LoftQのビット数を4に設定します。
# これは、モデルの量子化設定に関連します。
loftq_config = LoftQConfig(loftq_bits=4)

# LoraConfigを初期化し、モデルのトレーニング設定を指定します。
peft_config = LoraConfig(
    # target_modulesには、特定のターゲットモジュールを指定することができますが、ここではコメントアウトされています。
    # target_modules=['query_proj', 'value_proj', 'key_proj'],
    
    # RSLORAを使用するかどうかを指定します。ここでは使用することを選択しています。
    use_rslora=True,
    
    # LoRAのランクを設定します。この場合は16です。
    r=16,
    
    # LoRAのスケーリングファクターを指定します。この場合は8です。
    lora_alpha=8,
    
    # LoRAによるドロップアウト率を指定します。ここでは0.1です。
    lora_dropout=0.1,
    
    # タスクのタイプをSEQ_CLS（シーケンス分類）に設定します。
    task_type=TaskType.SEQ_CLS,
    
    # 初期LoRAウェイトの設定を'loftq'とします。
    init_lora_weights='loftq',
)

# モデルにPEFT設定を適用します。
# これにより、トレーニングのためにモデルが更新されます。
model = get_peft_model(model, peft_config)
```

</div>
</details>

In [ ]:
# LoftQConfigを使用して、LoftQのビット数を4に設定します。
# これは、モデルの量子化設定に関連します。
loftq_config = LoftQConfig(loftq_bits=4)

# LoraConfigを初期化し、モデルのトレーニング設定を指定します。
peft_config = LoraConfig(
    # target_modulesには、特定のターゲットモジュールを指定することができますが、ここではコメントアウトされています。
    # target_modules=['query_proj', 'value_proj', 'key_proj'],
    
    # RSLORAを使用するかどうかを指定します。ここでは使用することを選択しています。
    use_rslora=True,
    
    # LoRAのランクを設定します。この場合は16です。
    r=16,
    
    # LoRAのスケーリングファクターを指定します。この場合は8です。
    lora_alpha=8,
    
    # LoRAによるドロップアウト率を指定します。ここでは0.1です。
    lora_dropout=0.1,
    
    # タスクのタイプをSEQ_CLS（シーケンス分類）に設定します。
    task_type=TaskType.SEQ_CLS,
    
    # 初期LoRAウェイトの設定を'loftq'とします。
    init_lora_weights='loftq',
)

# モデルにPEFT設定を適用します。
# これにより、トレーニングのためにモデルが更新されます。
model = get_peft_model(model, peft_config)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
    
print_trainable_parameters(model)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルのトレーニング可能なパラメータを表示する関数を定義します。
# この関数は、トレーニング可能なパラメータの数と全パラメータの数、およびトレーニング可能な割合を計算して表示します。
def print_trainable_parameters(model):
    trainable_params = 0  # トレーニング可能なパラメータの初期値
    all_param = 0  # 全パラメータの初期値
    
    # モデルの各パラメータをイテレートします。
    for _, param in model.named_parameters():
        # 全パラメータの合計を更新します。
        all_param += param.numel()
        # パラメータがトレーニング可能な場合、そのカウントを増やします。
        if param.requires_grad:
            trainable_params += param.numel()
    
    # トレーニング可能なパラメータ、全パラメータ、トレーニング可能な割合を出力します。
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

# モデルのトレーニング可能なパラメータを表示するために関数を呼び出します。
print_trainable_parameters(model)
```

</div>
</details>

In [ ]:
# モデルのトレーニング可能なパラメータを表示する関数を定義します。
# この関数は、トレーニング可能なパラメータの数と全パラメータの数、およびトレーニング可能な割合を計算して表示します。
def print_trainable_parameters(model):
    trainable_params = 0  # トレーニング可能なパラメータの初期値
    all_param = 0  # 全パラメータの初期値
    
    # モデルの各パラメータをイテレートします。
    for _, param in model.named_parameters():
        # 全パラメータの合計を更新します。
        all_param += param.numel()
        # パラメータがトレーニング可能な場合、そのカウントを増やします。
        if param.requires_grad:
            trainable_params += param.numel()
    
    # トレーニング可能なパラメータ、全パラメータ、トレーニング可能な割合を出力します。
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

# モデルのトレーニング可能なパラメータを表示するために関数を呼び出します。
print_trainable_parameters(model)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Tokenize

</div>
<div class="column-right">

# 日本語訳

## トークナイズ

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
sep_token = tokenizer.sep_token_id
cls_token = tokenizer.cls_token_id
tokenizer.add_special_tokens({'pad_token': '<pad>'})
# model.resize_token_embeddings(len(tokenizer))
len_to_each = max_len // 3 - 3
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーからセパレータトークンとクラスストークンのIDを取得します。
sep_token = tokenizer.sep_token_id  # セパレータトークンのIDを取得
cls_token = tokenizer.cls_token_id  # クラスストークンのIDを取得

# トークナイザーにパディングトークンを追加します。
# ここでは、'<pad>'という文字列をパディングトークンとして追加します。
tokenizer.add_special_tokens({'pad_token': '<pad>'})

# モデルのトークン埋め込みのサイズをトークナイザーのトークン数の長さに合わせて調整することができますが、
# その行はコメントアウトされています。これを有効にすることで、トークン数に応じてモデルの埋め込み層が再調整されます。
# model.resize_token_embeddings(len(tokenizer))

# 各トークンの最大長を決定します。
# ここでは、max_lenを3で割って3を引き、各トークンが保持できる最大長を計算します。
len_to_each = max_len // 3 - 3
```

</div>
</details>

In [ ]:
# トークナイザーからセパレータトークンとクラスストークンのIDを取得します。
sep_token = tokenizer.sep_token_id  # セパレータトークンのIDを取得
cls_token = tokenizer.cls_token_id  # クラスストークンのIDを取得

# トークナイザーにパディングトークンを追加します。
# ここでは、'<pad>'という文字列をパディングトークンとして追加します。
tokenizer.add_special_tokens({'pad_token': '<pad>'})

# モデルのトークン埋め込みのサイズをトークナイザーのトークン数の長さに合わせて調整することができますが、
# その行はコメントアウトされています。これを有効にすることで、トークン数に応じてモデルの埋め込み層が再調整されます。
# model.resize_token_embeddings(len(tokenizer))

# 各トークンの最大長を決定します。
# ここでは、max_lenを3で割って3を引き、各トークンが保持できる最大長を計算します。
len_to_each = max_len // 3 - 3

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize_function(sample):
    prompt =  tokenizer(sample['prompt'], max_length=len_to_each, truncation=True, padding=True).input_ids
    response_a = tokenizer(sample['response_a'], max_length=len_to_each, truncation=True, padding=True).input_ids
    response_b = tokenizer(sample['response_b'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    sample['input_ids'] = [cls_token] + prompt + [sep_token] + response_a + [sep_token] + response_b
    return sample
```

</div>
<div class="column-right">

# 日本語訳

```python
# トークナイザーを使用して入力データをトークナイズする関数を定義します。
# この関数は、各サンプルのプロンプトと応答をトークン化し、モデルに適した形式に変換します。
def tokenize_function(sample):
    # プロンプトをトークナイズし、最大長len_to_eachに制限し、トランケーションとパディングを適用します。
    # ここでは、入力IDとしてトークンのIDリストを取得します。
    prompt = tokenizer(sample['prompt'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # 応答Aをトークナイズし、同様の処理を行います。
    response_a = tokenizer(sample['response_a'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # 応答Bをトークナイズし、同様の処理を行います。
    response_b = tokenizer(sample['response_b'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # トークンIDを結合し、モデルの入力形式を作成します。
    # ここでは、クラスストークン、プロンプト、セパレータトークン、応答A、セパレータトークン、応答Bを結合しています。
    sample['input_ids'] = [cls_token] + prompt + [sep_token] + response_a + [sep_token] + response_b
    return sample
```

</div>
</details>

In [ ]:
# トークナイザーを使用して入力データをトークナイズする関数を定義します。
# この関数は、各サンプルのプロンプトと応答をトークン化し、モデルに適した形式に変換します。
def tokenize_function(sample):
    # プロンプトをトークナイズし、最大長len_to_eachに制限し、トランケーションとパディングを適用します。
    # ここでは、入力IDとしてトークンのIDリストを取得します。
    prompt = tokenizer(sample['prompt'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # 応答Aをトークナイズし、同様の処理を行います。
    response_a = tokenizer(sample['response_a'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # 応答Bをトークナイズし、同様の処理を行います。
    response_b = tokenizer(sample['response_b'], max_length=len_to_each, truncation=True, padding=True).input_ids
    
    # トークンIDを結合し、モデルの入力形式を作成します。
    # ここでは、クラスストークン、プロンプト、セパレータトークン、応答A、セパレータトークン、応答Bを結合しています。
    sample['input_ids'] = [cls_token] + prompt + [sep_token] + response_a + [sep_token] + response_b
    return sample

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
dataset = dataset.map(tokenize_function)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 定義したtokenize_functionを使用して、dataset内の各サンプルに対してトークナイズ処理を適用します。
# このmap関数により、各サンプルがトークン化され、モデルに適した形式に変換されます。
dataset = dataset.map(tokenize_function)
```

</div>
</details>

In [ ]:
# 定義したtokenize_functionを使用して、dataset内の各サンプルに対してトークナイズ処理を適用します。
# このmap関数により、各サンプルがトークン化され、モデルに適した形式に変換されます。
dataset = dataset.map(tokenize_function)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
dataset
```

</div>
<div class="column-right">

# 日本語訳

```python
# 現在のdatasetデータセットを表示します。
# これにより、トークナイズ処理が適用された後のサンプルの構造を確認できます。
dataset
```

</div>
</details>

In [ ]:
# 現在のdatasetデータセットを表示します。
# これにより、トークナイズ処理が適用された後のサンプルの構造を確認できます。
dataset

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
dataset = dataset.select_columns(['input_ids', 'label'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# datasetから必要な列だけを選択し、新しいデータセットを作成します。
# ここでは、'input_ids'と'label'の2つの列を選択します。
dataset = dataset.select_columns(['input_ids', 'label'])
```

</div>
</details>

In [ ]:
# datasetから必要な列だけを選択し、新しいデータセットを作成します。
# ここでは、'input_ids'と'label'の2つの列を選択します。
dataset = dataset.select_columns(['input_ids', 'label'])

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Train

</div>
<div class="column-right">

# 日本語訳

## トレーニング

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
args = TrainingArguments(
    "LMSYS",
    eval_strategy = "steps",
    save_strategy = "steps",
    save_steps=500,
    learning_rate=3e-4,
    fp16=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    optim='adamw_hf',
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit = 2,
    metric_for_best_model='accuracy',
    report_to="wandb",
    run_name="DeBerta-base-train-data",
    eval_steps=500,
    logging_strategy="steps", 
    logging_steps=100,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングの引数を設定するためのTrainingArgumentsを初期化します。
args = TrainingArguments(
    "LMSYS",  # トレーニング結果の出力先ディレクトリ名
    eval_strategy="steps",  # 評価の実行戦略をステップごとに設定します。
    save_strategy="steps",  # モデルの保存戦略をステップごとに設定します。
    save_steps=500,  # 500ステップごとにモデルを保存します。
    learning_rate=3e-4,  # 学習率を設定します。
    fp16=True,  # 16ビット浮動小数点でトレーニングを行うかどうかを指定します。
    per_device_train_batch_size=4,  # 各デバイスのトレーニングバッチサイズを4に設定します。
    per_device_eval_batch_size=16,  # 各デバイスの評価バッチサイズを16に設定します。
    gradient_accumulation_steps=4,  # 勾配の累積ステップを4に設定します。
    optim='adamw_hf',  # 最適化アルゴリズムをAdamWに設定します。
    num_train_epochs=3,  # トレーニングエポック数を3に設定します。
    weight_decay=0.01,  # 重みの減衰を設定します。
    load_best_model_at_end=True,  # トレーニングの最後に最良のモデルをロードするかどうかを指定します。
    save_total_limit=2,  # 保存するモデルの最大数を2に設定します。
    metric_for_best_model='accuracy',  # 最良のモデルの評価基準を精度に設定します。
    report_to="wandb",  # 結果をWandBに報告します。
    run_name="DeBerta-base-train-data",  # 実行の名前を指定します。
    eval_steps=500,  # 評価を行うステップを500に設定します。
    logging_strategy="steps",  # ロギングの戦略をステップごとに指定します。
    logging_steps=100,  # 100ステップごとにログを出力します。
)
```

</div>
</details>

In [ ]:
# トレーニングの引数を設定するためのTrainingArgumentsを初期化します。
args = TrainingArguments(
    "LMSYS",  # トレーニング結果の出力先ディレクトリ名
    eval_strategy="steps",  # 評価の実行戦略をステップごとに設定します。
    save_strategy="steps",  # モデルの保存戦略をステップごとに設定します。
    save_steps=500,  # 500ステップごとにモデルを保存します。
    learning_rate=3e-4,  # 学習率を設定します。
    fp16=True,  # 16ビット浮動小数点でトレーニングを行うかどうかを指定します。
    per_device_train_batch_size=4,  # 各デバイスのトレーニングバッチサイズを4に設定します。
    per_device_eval_batch_size=16,  # 各デバイスの評価バッチサイズを16に設定します。
    gradient_accumulation_steps=4,  # 勾配の累積ステップを4に設定します。
    optim='adamw_hf',  # 最適化アルゴリズムをAdamWに設定します。
    num_train_epochs=3,  # トレーニングエポック数を3に設定します。
    weight_decay=0.01,  # 重みの減衰を設定します。
    load_best_model_at_end=True,  # トレーニングの最後に最良のモデルをロードするかどうかを指定します。
    save_total_limit=2,  # 保存するモデルの最大数を2に設定します。
    metric_for_best_model='accuracy',  # 最良のモデルの評価基準を精度に設定します。
    report_to="wandb",  # 結果をWandBに報告します。
    run_name="DeBerta-base-train-data",  # 実行の名前を指定します。
    eval_steps=500,  # 評価を行うステップを500に設定します。
    logging_strategy="steps",  # ロギングの戦略をステップごとに指定します。
    logging_steps=100,  # 100ステップごとにログを出力します。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return {
        'eval_log_loss': log_loss(labels, probabilities),
        'eval_accuracy': (np.argmax(logits, axis=1) == labels).mean()
    }
```

</div>
<div class="column-right">

# 日本語訳

```python
# 評価のためのメトリックを計算する関数を定義します。
# この関数は、モデルの出力とラベルを使用して評価指標を計算します。
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # 評価予測からロジットとラベルを取得します。
    
    # ソフトマックス関数を適用して、ロジットを確率に変換します。
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    
    # 評価指標を辞書形式で返します。
    return {
        'eval_log_loss': log_loss(labels, probabilities),  # ログロスを計算します。
        'eval_accuracy': (np.argmax(logits, axis=1) == labels).mean()  # 精度を計算します。
    }
```

</div>
</details>

In [ ]:
# 評価のためのメトリックを計算する関数を定義します。
# この関数は、モデルの出力とラベルを使用して評価指標を計算します。
def compute_metrics(eval_pred):
    logits, labels = eval_pred  # 評価予測からロジットとラベルを取得します。
    
    # ソフトマックス関数を適用して、ロジットを確率に変換します。
    probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    
    # 評価指標を辞書形式で返します。
    return {
        'eval_log_loss': log_loss(labels, probabilities),  # ログロスを計算します。
        'eval_accuracy': (np.argmax(logits, axis=1) == labels).mean()  # 精度を計算します。
    }

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
data_collator = DataCollatorWithPadding(tokenizer)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データのバッチ処理を行うためのDataCollatorWithPaddingを初期化します。
# これにより、入力シーケンスが同じ長さにパディングされ、モデルへの入力形式が整えられます。
data_collator = DataCollatorWithPadding(tokenizer)
```

</div>
</details>

In [ ]:
# データのバッチ処理を行うためのDataCollatorWithPaddingを初期化します。
# これにより、入力シーケンスが同じ長さにパディングされ、モデルへの入力形式が整えられます。
data_collator = DataCollatorWithPadding(tokenizer)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングを実行するためのTrainerを初期化します。
trainer = Trainer(
    model=model,  # トレーニングするモデルを指定します。
    args=args,  # 設定したトレーニング引数を指定します。
    train_dataset=dataset['train'],  # トレーニングデータセットを指定します。
    eval_dataset=dataset['test'],  # 評価データセットを指定します。
    compute_metrics=compute_metrics,  # 評価指標を計算する関数を指定します。
    data_collator=data_collator,  # データのバッチ処理を行うコラレーターを指定します。
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 早期停止のコールバックを設定します。
)
```

</div>
</details>

In [ ]:
# トレーニングを実行するためのTrainerを初期化します。
trainer = Trainer(
    model=model,  # トレーニングするモデルを指定します。
    args=args,  # 設定したトレーニング引数を指定します。
    train_dataset=dataset['train'],  # トレーニングデータセットを指定します。
    eval_dataset=dataset['test'],  # 評価データセットを指定します。
    compute_metrics=compute_metrics,  # 評価指標を計算する関数を指定します。
    data_collator=data_collator,  # データのバッチ処理を行うコラレーターを指定します。
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 早期停止のコールバックを設定します。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer.train()
wandb.finish()
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニングを開始します。
# これにより、指定された設定に従ってモデルのトレーニングが実行されます。
trainer.train()

# トレーニングが完了したら、WandBのログを終了します。
# これにより、現在の実行の情報が保存されます。
wandb.finish()
```

</div>
</details>

In [ ]:
# トレーニングを開始します。
# これにより、指定された設定に従ってモデルのトレーニングが実行されます。
trainer.train()

# トレーニングが完了したら、WandBのログを終了します。
# これにより、現在の実行の情報が保存されます。
wandb.finish()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
save_folder = 'DeBerta-base-trained'
model = model.merge_and_unload()
model.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)
```

</div>
<div class="column-right">

# 日本語訳

```python
# トレーニング済みモデルとトークナイザーを保存するためのフォルダー名を設定します。
save_folder = 'DeBerta-base-trained'

# モデルをマージしてメモリからアンロードします。
# これにより、モデルの重みが最適化され、次のステップで保存できる状態になります。
model = model.merge_and_unload()

# 保存先フォルダーにモデルの重みを保存します。
model.save_pretrained(save_folder)

# 保存先フォルダーにトークナイザーの設定を保存します。
tokenizer.save_pretrained(save_folder)
```

</div>
</details>

In [ ]:
# トレーニング済みモデルとトークナイザーを保存するためのフォルダー名を設定します。
save_folder = 'DeBerta-base-trained'

# モデルをマージしてメモリからアンロードします。
# これにより、モデルの重みが最適化され、次のステップで保存できる状態になります。
model = model.merge_and_unload()

# 保存先フォルダーにモデルの重みを保存します。
model.save_pretrained(save_folder)

# 保存先フォルダーにトークナイザーの設定を保存します。
tokenizer.save_pretrained(save_folder)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
trainer.save_model(f"{save_folder}-trainer")
```

</div>
<div class="column-right">

# 日本語訳

```python
# Trainerオブジェクトを使用して、トレーニング済みモデルを指定したフォルダーに保存します。
# ここでは、フォルダー名に"-trainer"を追加して保存先を指定します。
trainer.save_model(f"{save_folder}-trainer")
```

</div>
</details>

In [ ]:
# Trainerオブジェクトを使用して、トレーニング済みモデルを指定したフォルダーに保存します。
# ここでは、フォルダー名に"-trainer"を追加して保存先を指定します。
trainer.save_model(f"{save_folder}-trainer")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

```

</div>
<div class="column-right">

# 日本語訳

```python
# コードセルが空のため、何も実行しません。必要に応じてコードを記述してください。
```

</div>
</details>

In [ ]:
# コードセルが空のため、何も実行しません。必要に応じてコードを記述してください。